# Worksheet: SVD for recommendation system

Please read section 2.3 in this [notes](https://math.ucsd.edu/sites/math.ucsd.edu/files/undergrad/honors-program/honors-theses/2017-2018/Zecheng_Kuang_Honors_Thesis.pdf), then transfer it to python code. 

You should write a class to do this. This class should include initialization, fit method that computes SVD based on given data matrix, and recommendation to new users (notice that there are two recommendation methods in the notes, so you should write two methods in your class).

In the notes, one new user is given, but your recommendation method should work for multiple new users (matrix multiplication helps).

After writting the codes, testing the correctness using the data given in the notes. You should design the test examples. The instructions are given below.

In [1]:
import numpy as np
from numpy.linalg import svd
from scipy.spatial.distance import cosine

class RecommendationSystem:
    def __init__(self, ratings_matrix):
        # Perform Singular Value Decomposition
        self.ratings_matrix = ratings_matrix
        self.U, self.S, self.Vt = svd(ratings_matrix, full_matrices=False)
        
        # Reduce dimensions if necessary (only retain the top k singular values)
        self.S_diag = np.diag(self.S)
    
    def map_users_to_genre_space(self):
        # Map users to "genre space" using the U and S matrices
        user_genre_matrix = self.U @ self.S_diag
        return user_genre_matrix
    
    def map_new_user_to_genre_space(self, new_user_ratings):
        # Map a new user's ratings to "genre space"
        user_genre_representation = new_user_ratings @ self.Vt.T
        return user_genre_representation

    def cosine_similarity(self, vec1, vec2):
        # Calculate cosine similarity
        return 1 - cosine(vec1, vec2)

    def recommend_based_on_similar_users(self, new_user_ratings, top_n=1):
        # Map users and the new user to genre space
        user_genre_matrix = self.map_users_to_genre_space()
        new_user_genre_vector = self.map_new_user_to_genre_space(new_user_ratings)
        
        # Calculate similarity between the new user and all existing users
        similarities = [self.cosine_similarity(new_user_genre_vector, user_vec)
                        for user_vec in user_genre_matrix]
        
        # Find the most similar users
        similar_user_indices = np.argsort(similarities)[-top_n:]
        
        # Aggregate recommendations from similar users
        recommendations = np.zeros(self.ratings_matrix.shape[1])
        for idx in similar_user_indices:
            recommendations += self.ratings_matrix[idx]
        
        return recommendations

In [2]:
# test example 1: 
# representing Jane's rating as a vector u = [0, 0, 4, 0, 0], then mapping her ratings into the “genre space”
# User ratings matrix
ratings_matrix = np.array([
    [1, 1, 1, 0, 0],
    [3, 3, 3, 0, 0],
    [4, 4, 4, 0, 0],
    [5, 5, 5, 0, 0],
    [0, 0, 0, 4, 4],
    [0, 0, 0, 5, 5],
    [0, 0, 0, 2, 2]
])

# Initialize recommendation system with ratings
rec_sys = RecommendationSystem(ratings_matrix)

# New user (Jane) only rated Star Wars with a 4
new_user_ratings = np.array([0, 0, 4, 0, 0])
jane_genre_vector = rec_sys.map_new_user_to_genre_space(new_user_ratings)
print("Jane's genre space representation:", np.round(jane_genre_vector, 2))


Jane's genre space representation: [-2.31  0.    0.   -1.63  2.83]


In [3]:
# test example 2: find users who are similar to Jane and 
# provide recommendations to Jane according to the preferences of those similar users.
recommendations = rec_sys.recommend_based_on_similar_users(new_user_ratings, top_n=1)
print("Recommendations for Jane based on similar users:", np.round(recommendations, 2))


Recommendations for Jane based on similar users: [4. 4. 4. 0. 0.]
